# OpenGridMap Notebook  -- Algorithm Inference

# Step1. Data Preparation

## Import necessary modules

In [20]:
import psycopg2
import json
import ipyleaflet as L
import networkx as nx
from ipyleaflet import (
    Map,
    Marker,
    TileLayer, ImageOverlay,
    Polyline, Polygon, Rectangle, Circle, CircleMarker,
    GeoJSON,
    DrawControl
)

## Database connection

In [21]:
# Set connection to gis db
try:
    conn = psycopg2.connect("dbname=gis user=jennyzhou")
except:
    print 'Fail to connect to Postgres Server'

## Select transformer, house within one vornoi area

In [31]:
cur = conn.cursor()
vid = 38
# Select vornoi polygon and transformer
cur.execute("SELECT ST_AsGeoJSON((ST_FlipCoordinates(ST_Transform(way, 4326)))), "
            "ST_AsGeoJSON((ST_FlipCoordinates(ST_Transform(tway, 4326)))) FROM vornoi where vid = %d ;" % vid)
data = cur.fetchall()

# Select all the houses
cur.execute("SELECT ST_AsGeoJSON(ST_Collect(ST_FlipCoordinates(ST_Transform(way, 4326)))) "
            "FROM vornoi_map where vid = %d and istransformer = false ;" % vid)
houses = cur.fetchall()
cur.close()

vornoiJson = json.loads(data[0][0])
transJson = json.loads(data[0][1])
houseJson = json.loads(houses[0][0])

# Step2. Data visualisation with ipyleaflet

In [32]:
# Mark transformer
tCenter = transJson['coordinates']
tMarker =  Marker(location=tCenter)

# Mark polygon
tPoly = Polygon(locations=vornoiJson['coordinates'], weight=2,
            color='#a3c2c2', opacity=0.8, fill_opacity=0.2,
            fill_color='#ccffcc') 
    
# Init Map
tMap = Map(default_tiles=TileLayer(opacity=1.0),center=tCenter ,zoom=16)
tMap.add_layer(tMarker)
tMap.add_layer(tPoly)
tMap

In [35]:
# Mark houses
hmarkers = []
for hp in houseJson['coordinates']:
    c = Circle(location= hp, weight=5, opacity = 0.5, color = 'red', radius = 2) 
    hmarkers.append(c)
    tMap.add_layer(c) 

In [34]:
for mk in hmarkers:
    tMap.remove_layer(mk)

# Step3. Construct network graph

## Preparing vertices and edges

In [55]:
cur = conn.cursor()
# Select vornoi polygon and transformer
cur.execute("SELECT t1.oid, t2.oid, ST_Distance(t1.way, t2.way) "
            "FROM (SELECT oid, way FROM vornoi_map WHERE vid = %d) AS t1, "
            "(SELECT oid, way FROM vornoi_map WHERE vid = %d) AS t2 WHERE t1.oid != t2.oid;"% (vid,vid))
nodes = cur.fetchall()
cur.close()

## Build graph with data set

In [56]:
graph = []
for ele in nodes:
    graph.append("%d %d %f" % ele)

G = nx.parse_edgelist(graph, nodetype = int, data=(('weight',float),))

In [60]:
MST = nx.minimum_spanning_tree(G)
type(MST.edges()[0][1])


int

In [97]:
conn.close()
conn = psycopg2.connect("dbname=gis user=jennyzhou")



## Minimum Spanning Tree

In [95]:
# To be implemented
MST = nx.minimum_spanning_tree(G)

# Storing minimum spanning tree into database
cur = conn.cursor()
cur.execute("CREATE TABLE IF NOT EXISTS templines (way geometry(LineString,900913));")

for edge in MST.edges():
    (start, end) = edge
    cur.execute("INSERT INTO templines (SELECT ST_MakeLine(t1.way, t2.way) "
                "FROM (SELECT oid, way FROM vornoi_map WHERE vid = %d) AS t1, "
                "(SELECT oid, way FROM vornoi_map WHERE vid = %d) AS t2 "
                "WHERE t1.oid = %d AND t2.oid = %d);"% (vid, vid, start, end ))
    conn.commit()   
    
cur.execute("INSERT INTO graph (vid, type, way) SELECT %d, %s, ST_Collect(way) FROM templines;"%(vid, '\'Mst\''))
conn.commit()
cur.close()

# Step4. Display graph on map

In [102]:
cur = conn.cursor()
cur.execute("SELECT ST_AsGeoJSON(ST_FlipCoordinates(ST_Transform(way, 4326)))"
            "FROM graph WHERE vid = %d;" %vid)
lines = cur.fetchall()
lineJson = json.loads(lines[0][0])

In [114]:
lmarkers = []
for l in lineJson['coordinates']:
    pl = Polyline(locations=l, weight=2, color='#2929a3', opacity=0.5)
    lmarkers.append(pl)
    tMap.add_layer(pl)
tMap

In [113]:
for mk in lmarkers:
    tMap.remove_layer(mk)